<a href="https://colab.research.google.com/github/Seysande/nle-prediction/blob/main/nle_evaluation_XGBoost_SMOTE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Data and Packages

In [ ]:
%reset

In [ ]:
!pip install rdkit-pypi
!pip install bayesian-optimization

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
import xgboost as xgb
import seaborn as sns

# Data Preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.base import BaseEstimator, TransformerMixin
from imblearn.pipeline import Pipeline

# Evaluation
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve, ConfusionMatrixDisplay, confusion_matrix
from sklearn.metrics import precision_recall_curve, auc, precision_score, recall_score, average_precision_score
from bayes_opt import BayesianOptimization
from imblearn.over_sampling import SMOTE

df = pd.read_csv('drugData_jan-2024.csv')
smiles_list = df['smiles'].tolist()

In [ ]:
print(xgb.__version__)

In [ ]:
def smiles_to_mol(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            raise ValueError(f"Invalid SMILES string: {smiles}")
        return mol
    except Exception as e:
        print(e)
        return None

molecules = [smiles_to_mol(smiles) for smiles in smiles_list]
molecules = [mol for mol in molecules if mol is not None]
fingerprints = [AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048) for mol in molecules]
df['fingerprint'] = fingerprints
df = df.drop('smiles', axis=1)

X = np.array(list(df['fingerprint']))
y = df['label']
y = np.array(y)

np.shape(X)
display(X)

# Training and Optimization

In [ ]:
def xgb_eval(n_estimators, eta, max_depth, min_child_weight, scale_pos_weight, subsample, colsample_bytree, max_delta_step, X_train, y_train, X_test, y_test):
  n_estimators = int(n_estimators)
  max_depth = int(max_depth)
  subsample = max(0, min(1, subsample))
  colsample_bytree = max(0.5, min(1, colsample_bytree))
  min_child_weight = int(min_child_weight)

  # Defining a dictionary of parameters
  params = {
      'n_estimators': n_estimators,
      'eta': eta,
      'max_depth': max_depth,
      'min_child_weight': min_child_weight,
      'scale_pos_weight': scale_pos_weight,
      'subsample': subsample,
      'colsample_bytree': colsample_bytree,
      'max_delta_step': max_delta_step,
      'eval_metric':'aucpr',
      'objective': 'binary:logistic',
      'use_label_encoder':False,
  }
  # Set up parameters for XGBoost
  model = xgb.XGBClassifier(**params)
  model.fit(X_train, y_train)
  y_pred_proba = model.predict_proba(X_test)[:, 1]

  precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
  pr_auc = auc(recall, precision)

  return pr_auc

#
param_bounds = {
    'n_estimators': (50, 1000),
    'eta': (0.01, 0.1),
    'max_depth': (3, 7),
    'min_child_weight': (1, 10),
    'scale_pos_weight':(9, 11),
    'subsample': (0.6, 0.9),
    'colsample_bytree': (0.6, 0.9),
    'max_delta_step': (0, 1),
}

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

precision_values = []
recall_values = []
average_precisions = []
fprs = []
tprs = []
confusion_matrices = []
accuracies = []

for fold_idx, (train_index, test_index) in enumerate(kf.split(X, y)):
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]


  smote = SMOTE(random_state=42)
  X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Take x_train, y_train and split 80/20
  X_train_res, X_val, y_train_res, y_val = train_test_split(X_train_res, y_train_res, test_size = 0.2, random_state = 42)

  def xgb_eval_curr(n_estimators, eta, min_child_weight, scale_pos_weight, max_depth, max_delta_step, subsample, colsample_bytree):
    return xgb_eval(n_estimators, eta, min_child_weight, scale_pos_weight, max_depth, max_delta_step, subsample, colsample_bytree, X_train_res, y_train_res, X_test, y_test)

# Hyperparameter Optimization
  optimizer = BayesianOptimization(f=xgb_eval_curr, pbounds=param_bounds, random_state=42, verbose=1)
  optimizer.maximize(init_points=16, n_iter=5)

# Store best parameters
  best_params = optimizer.max['params']
  best_params['max_depth'] = int(best_params['max_depth'])
  best_params['n_estimators'] = int(best_params['n_estimators'])
  best_params['min_child_weight'] = int(best_params['min_child_weight'])
  best_params['subsample'] = max(0, min(1, best_params['subsample']))
  best_params['colsample_bytree'] = max(0.5, min(1, best_params['colsample_bytree']))


  dtrain = xgb.DMatrix(X_train, label=y_train)
  dtest = xgb.DMatrix(X_test, label=y_test)
  num_rounds=100

  bst = xgb.train(best_params, dtrain, num_rounds)
  y_pred_prob = bst.predict(dtest)
  y_pred = (y_pred_prob > 0.5).astype(int)

  precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
  average_precision = average_precision_score(y_test, y_pred_prob)

  fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
  roc_auc = roc_auc_score(y_test, y_pred_prob)

  # AUC values for each graph
  fprs.append(np.interp(np.linspace(0, 1, 100), fpr, tpr))
  tprs.append(np.linspace(0, 1, 100))

  # Store PR values for the graph
  precision_values.append(np.interp(np.linspace(0, 1, 100), recall[::-1], precision[::-1]))
  recall_values.append(np.linspace(0, 1, 100))
  average_precisions.append(average_precision)

  cm = confusion_matrix(y_test, y_pred)
  confusion_matrices.append(cm)


  accuracy = accuracy_score(y_test, y_pred)
  accuracies.append(accuracy)

  plt.plot(recall, precision, lw=2, label=f'Fold {fold_idx+1} (area = {average_precision:.2f})')


mean_confusion_matrix = np.mean(confusion_matrices, axis=0)
plt.figure(figsize=(8, 6))
sns.heatmap(mean_confusion_matrix, annot=True, fmt=".2f", cmap="Blues", xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.title('Mean Confusion Matrix')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.show()



mean_precision = np.mean(precision_values, axis=0)
mean_recall = np.mean(recall_values, axis=0)
mean_fpr = np.mean(fprs, axis=0)
mean_tpr = np.mean(tprs, axis=0)

mean_accuracy = np.mean(accuracies)
mean_average_precision = np.mean(average_precision)
print(f"Mean Average Precision: {mean_average_precision:.2f}")
print(f"Mean Accuracy: {mean_accuracy:.2f}")

plt.plot(mean_recall, mean_precision, 'k--', lw=2, label = f'Mean (area = {mean_average_precision:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend(loc="lower left")
plt.show()




In [ ]:
plt.plot(mean_tpr, mean_fpr, lw=2, label=f'Mean (area = {mean_accuracy:.2f})')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right")
plt.show()


plt.plot(mean_recall, mean_precision, lw=2, color="purple", label = f'Mean (area = {mean_average_precision:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim(0, 1)
plt.legend(loc="lower left")
plt.show()